# Oxfordpet dataset 생성
1. 데이터 파일 다운로드
2. 압축 풀기
3. 데이터파일 경로 목록 생성
4. 전처리 -> jpg color를 제외한 이미지는 삭제
5. y값 정의 -> 파일명을 이용해서 각 이미지들의 품종값을 추출해서 y값 정의
6. train/validation/test set 분리
7. Dataset 정의

In [1]:
import os
import re
from glob import glob
import tarfile
from PIL import Image

import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

In [2]:
#1. 데이터 파일 다운로드
import os

img_tarfile_path = r"C:\classes\classes\y_deeplearning\images.tar.gz"
pet_data_path = r"C:\classes\classes\y_deeplearning\datasets"
pet_image_path = os.path.join(pet_data_path, "images")

In [6]:
#2. 압축 풀기
with tarfile.open(img_tarfile_path) as tar:
    tar.extractall(pet_data_path)

In [3]:
#3. 데이터 파일 경로 목록 생성
file_list = glob(pet_image_path+"\\*.jpg")
file_list[:5]

['C:\\classes\\classes\\y_deeplearning\\datasets\\images\\Abyssinian_1.jpg',
 'C:\\classes\\classes\\y_deeplearning\\datasets\\images\\Abyssinian_10.jpg',
 'C:\\classes\\classes\\y_deeplearning\\datasets\\images\\Abyssinian_100.jpg',
 'C:\\classes\\classes\\y_deeplearning\\datasets\\images\\Abyssinian_101.jpg',
 'C:\\classes\\classes\\y_deeplearning\\datasets\\images\\Abyssinian_102.jpg']

In [8]:
#4. 전처리 -> jpg color 가 아닌 이미지 파일을 삭제
remove_cnt = 0
for file in file_list:
    with Image.open(file) as img:
        image_mode = img.mode
    if image_mode != "RGB":
        os.remove(file)
        remove_cnt += 1
        print(f"삭제파일명 : {os.path.basename(file)}, mode: {image_mode}")

In [9]:
#4_1. 이미지 파일 목록을 갱신
file_list = glob(pet_image_path+"\\*.jpg")
print(len(file_list))

7378


In [18]:
#5. y값 정의 -> 파일명을 이용해 각 이미지들의 class 추출 후 y값 정의
import re


## 1) 파일명을 이용해서 y값 정의 / 중복제거를 위해 set 사용
class_name_set = set()
for file in file_list:
    file_name = os.path.basename(file) 
    file_name = os.path.splitext(file_name)[0]
    label = re.sub('_\d+', "", file_name) # \d+ 정수 1개이상
    class_name_set.add(label)

## 2) y값 리스트 반환 후 정렬
index_to_class = list(class_name_set)
index_to_class.sort()


class_to_index = {value:index for index, value in enumerate(index_to_class)}

In [19]:
#6. train / valid / test set 분리 -> file 목록 index로 분리
# 200 장 기준, train 70%, valid 20%, test 10%

train_idx = int(200 * 0.7)
val_idx = train_idx + int(200 * 0.2)

#6-1. train / validation / test set 에 넣을 파일경로 나누기.
train_file_list = []
valid_file_list = []
test_file_list = []
cnt = 0 #class별 파일 index
previous_class = None # 이전 처리했던 파일이 어느 class 인지 저장할 변수.
file_list.sort() # file_list 정렬 - 같은 품종의 파일끼리 뭉쳐있도록 정렬.

for path in file_list:
    file_name = os.path.splitext(os.path.basename(path))[0] # 디렉토리빼고 확장자빼고 파일명만 추출
    class_name = re.sub(r"_\d+", "", file_name) # 품종만 추출

    if previous_class == class_name: # 이전 처리 class와 현재 처리 class가 같은지 확인
        cnt += 1
    else:
        cnt = 1
    
    if cnt <= train_idx: # 0~ 140
        train_file_list.append(path)
    elif cnt > train_idx and cnt <=val_idx: # 140 ~ 180
        valid_file_list.append(path)
    else: # 180~200
        test_file_list.append(path)
    
    previous_class = class_name # 현재 처리한 class 이름을 previous_class이름으로 설정

In [20]:
#7. Dataset 구현
import cv2

class OxfordPetDataset(Dataset):
    def __init__(self, path_list, class_to_index, index_to_class, transform=None):
        self.path_list = path_list
        self.transform = transform
        self.class_to_index = class_to_index
        self.index_to_class = index_to_class

    def __len__(self):
        return len(self.path_list)
    
    def __getitem__(self, index):
        path = self.path_list[index]
        img = cv2.cvtcolor(cv2.imread(path), cv2.COLROR_BGR2RGB)
        img = cv2.resize(img, (224, 224))

        if self.transform is not None:
            img = self.transform(img)
        class_name = re.sub(r"_\d+", "", os.path.basename(path))
        class_index = self.class_to_index[class_name]
        return img, class_index

In [22]:
# 7-1. dataset 정의
trainset = OxfordPetDataset(
    train_file_list,
    class_to_index,
    index_to_class
)

validset = OxfordPetDataset(valid_file_list, class_to_index, index_to_class)
testset = OxfordPetDataset(test_file_list, class_to_index, index_to_class)

train_loader = DataLoader(trainset, batch_size=32, shuffle=True)
valid_loader = DataLoader(validset, batch_size=32)
test_loader = DataLoader(testset, batch_size=32)